In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_params, utils
from phoenix.tag.topic import single_feature_match_topic_config as sfm_topic_config
from phoenix.tag.labelling import backport_sflm_to_sfm, sflm_processing
from phoenix.tag.labelling import utils as labelling_utils

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None
TENANT_ID = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Note: this is a list of all `object_type`s the tenant has an sflm for.
OBJECT_TYPES = ["facebook_posts"]


In [ ]:
cur_run_params = run_params.general.create(ARTIFACTS_ENVIRONMENT_KEY, TENANT_ID, RUN_DATETIME)

# INPUT
sflm_url_dict = {}
for OBJECT_TYPE in OBJECT_TYPES:
    url_config = {"OBJECT_TYPE": OBJECT_TYPE}
    url = cur_run_params.art_url_reg.get_url("sflm-single_object_type", url_config)
    sflm_url_dict[OBJECT_TYPE] = url
    

# OUTPUT
SFM_TOPIC_CONFIG_URL = cur_run_params.art_url_reg.get_url("static-legacy-sfm-config")



In [ ]:
# Display params.
print(
cur_run_params.run_dt.dt,
cur_run_params.tenant_config,
sflm_url_dict,
SFM_TOPIC_CONFIG_URL,
sep='\n',
)

In [ ]:
sflm_df = pd.DataFrame()
for url in sflm_url_dict.values():
    df = artifacts.dataframes.get(url).dataframe
    sflm_df = sflm_df.append(df)

In [ ]:
sfm_df = backport_sflm_to_sfm.sflm_to_sfm(sflm_df)

In [ ]:
with tentaclio.open(SFM_TOPIC_CONFIG_URL, "w") as fh:
    sfm_df.to_csv(fh, index=False)